<a href="https://colab.research.google.com/github/wiv33/A-Learning-python/blob/master/machine-learning/_000_hello_machine/_000_basic/_003_cuk_edu/_004_3_text_processing/_004_imdb_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from tensorflow.keras import datasets
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, BatchNormalization, SpatialDropout1D, Activation
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

vocab_size = 10000  # final field
(X_train, y_train), (X_test, y_test) = datasets.imdb.load_data(num_words = vocab_size)
print(X_train.shape)
#0th sentence

print(len(X_train[0]))

#1th sentence
print(len(X_train[1]))

max_len = 200
X_train = pad_sequences(X_train, maxlen = max_len)
X_test = pad_sequences(X_test, maxlen = max_len)

print('X_train의 크기(shape) :',X_train.shape)
print('X_test의 크기(shape) :',X_test.shape)
print(y_train[:5])


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

(25000,)
218
189
X_train의 크기(shape) : (25000, 200)
X_test의 크기(shape) : (25000, 200)
[1 0 0 1 0]


# 모델 결과 출력

In [17]:
def exec(m, is_subclass = False):
  model_name = m.name 
  ext = '' if is_subclass else '.h5'
  es = EarlyStopping(monitor='val_loss', 
                   mode='min', 
                   verbose = 1, 
                   patience = 3)

  mc = ModelCheckpoint(f'{model_name}{ext}', 
                      monitor='val_acc', 
                      mode='max', 
                      verbose=1, 
                      save_best_only=True,
                      save_weights_only=is_subclass,
                      save_format='tf' if is_subclass else 'h5')
  
  m.compile(optimizer=optimizers.Adam(1e-3), 
              loss='binary_crossentropy',
              metrics=['acc'])

  history = m.fit(X_train,
                  y_train,
                  epochs=3, 
                  validation_data=(X_test, y_test), 
                  callbacks=[es, mc])
  loaded_model = load_model(f'{model_name}{ext}')
  
  print("\n %s의 테스트 정확도: %.4f" % (model_name, loaded_model.evaluate(X_test, y_test)[1]))


# RNN 모델

In [19]:
class RNNClassifier(tf.keras.Model):
  def __init__(self, vocab_size, max_seq_len, **kargs):
    super(RNNClassifier, self).__init__(**kargs)
    self.embedding = layers.Embedding(input_dim=vocab_size, output_dim=max_seq_len)
    self.lstm_1_layer = layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
    self.lstm_2_layer = layers.Bidirectional(tf.keras.layers.LSTM(64))
    self.dropout = layers.Dropout(0.3)
    self.fc1 = layers.Dense(units=128)
    self.batch = layers.BatchNormalization()
    self.act1 = layers.Activation(activation='tanh')
    self.fc2 = layers.Dense(units=1, activation=tf.keras.activations.sigmoid)

  def call(self, x):
    x = self.embedding(x)
    x = self.dropout(x)
    x = self.lstm_1_layer(x)
    x = self.lstm_2_layer(x)
    x = self.dropout(x)
    x = self.fc1(x)
    x = self.batch(x)
    x = self.act1(x)
    x = self.dropout(x)
    x = self.fc2(x)
    return x


# model = RNNClassifier(vocab_size, max_len)
# model.build(input_shape=(128, max_len))

model = Sequential()
model.add(Embedding(vocab_size, 256))
model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(layers.Dropout(0.3))
model.add(Dense(128))
model.add(layers.BatchNormalization())
model.add(layers.Activation('tanh'))
model.add(Dense(1, activation=tf.keras.activations.sigmoid))

model.summary()
exec(model, False)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 256)         2560000   
_________________________________________________________________
bidirectional_16 (Bidirectio (None, None, 128)         164352    
_________________________________________________________________
bidirectional_17 (Bidirectio (None, 128)               98816     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               16512     
_________________________________________________________________
batch_normalization_8 (Batch (None, 128)               512       
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0

# CNN + LSTM

In [22]:
# 예측 모델 CNN 또는 RNN 사용 가능

# model = Sequential()
# model.add(Embedding(vocab_size, 256))
# model.add(SpatialDropout1D(0.3))
# model.add(Conv1D(256, 3, padding='valid', activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(128, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))



class SentimentAnalysisModel(tf.keras.Model):
  def __init__(self, vocab_size, max_seq_len, **kwargs):
    super(SentimentAnalysisModel, self).__init__(name = 'SentimentAnalysis', **kwargs)
    self.embedding = layers.Embedding(vocab_size, max_seq_len)
    self.drop = layers.SpatialDropout1D(0.3)
    
    self.cv1 = layers.Conv1D(256, 3, padding='valid')
    self.act1 = layers.Activation(activation='swish')
    self.bat1 = layers.BatchNormalization()

    self.ds = layers.Dense(128)
    self.bat2 = layers.BatchNormalization()
    self.act2 = layers.Activation(activation='swish')

    self.bilstm = layers.Bidirectional(layers.LSTM(max_seq_len))
    self.ds2 = layers.Dense(64, activation='relu')
    self.out = layers.Dense(1, activation='sigmoid')

  def call(self, x):
    x = self.embedding(x)
    x = self.drop(x)

    x = self.cv1(x)
    x = self.bat1(x)
    x = self.act1(x)

    x = self.ds(x)
    x = self.bat2(x)
    x = self.act2(x)

    x = self.bilstm(x)
    x = self.ds2(x)
    x = self.out(x)
    return x

# model = SentimentAnalysisModel(vocab_size, max_len)
# model.build(input_shape=(128, max_len))

model = models.Sequential()
model.add(Embedding(vocab_size, max_len))
model.add(layers.SpatialDropout1D(0.3))
model.add(layers.Conv1D(256, 3, padding='valid'))
model.add(layers.BatchNormalization())
model.add(layers.Activation('swish'))
model.add(layers.Dense(128))
model.add(layers.BatchNormalization())
model.add(layers.Activation('swish'))
model.add(layers.Bidirectional(layers.LSTM(max_len)))
model.add(Dense(1, activation='sigmoid'))


model.summary()




# 모델 실행
exec(model)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 200)         2000000   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, None, 200)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 256)         153856    
_________________________________________________________________
batch_normalization_11 (Batc (None, None, 256)         1024      
_________________________________________________________________
activation_11 (Activation)   (None, None, 256)         0         
_________________________________________________________________
dense_19 (Dense)             (None, None, 128)         32896     
_________________________________________________________________
batch_normalization_12 (Batc (None, None, 128)        

# CNN 모델

In [25]:
inputs = layers.Input(shape=(None,), dtype='int64')

embedded_sequences = layers.Embedding(vocab_size, 256)(inputs)

x = layers.Dropout(0.3)(embedded_sequences)
x = layers.Conv1D(256, 3, padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling1D(7)(x)

x = layers.Conv1D(256, 3, padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling1D(7)(x)

x = layers.Conv1D(256, 3, padding='valid')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.GlobalMaxPool1D()(x)

x = layers.Dense(128)(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)

x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation='sigmoid')(x)

model = models.Model(inputs, outputs)

model.summary()

# 모델 실행
exec(model)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_13 (Embedding)     (None, None, 256)         2560000   
_________________________________________________________________
dropout_12 (Dropout)         (None, None, 256)         0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, None, 256)         196864    
_________________________________________________________________
batch_normalization_20 (Batc (None, None, 256)         1024      
_________________________________________________________________
activation_19 (Activation)   (None, None, 256)         0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, None, 256)         0   